In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz, process
%matplotlib inline

C:\Users\asbel.hernandez\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\asbel.hernandez\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\asbel.hernandez\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\asbel.hernandez\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\asb

In [56]:
s1 = "Via Aurelio Saffi, 26 20123 Milano"
s2 = "Via Aurelio Saffi 26 Milano"
s3 = "Via A. Saffi 26 Milano"
s4 = "Via Saffi Aurelio 26 Milano"
s5 = "Milano Via A. Saffi 26"
s6 = "Via Aurelio Saffi numero civico 26 Milano"
s7 = "Via Aurelio Saffi, 36 20123 Milano"
s8 = "Via A. Maffi, 26 20123 Milano"
options = []
options = np.append(np.asarray(options), [s1, s2, s3, s4, s5, s6, s7, s8])
options

array(['Via Aurelio Saffi, 26 20123 Milano',
       'Via Aurelio Saffi 26 Milano', 'Via A. Saffi 26 Milano',
       'Via Saffi Aurelio 26 Milano', 'Milano Via A. Saffi 26',
       'Via Aurelio Saffi numero civico 26 Milano',
       'Via Aurelio Saffi, 36 20123 Milano',
       'Via A. Maffi, 26 20123 Milano'], dtype='<U41')

In [61]:
# s1 = "Via Aurelio Saffi, 26 20123 Milano"
levenshtein = process.extract(s1, options, scorer=fuzz.ratio, limit=7)
strings = []
ratios_levenshtein = []
for index, row in enumerate(levenshtein):
    strings = np.append(strings, row[0]) 
    ratios_levenshtein = np.append(ratios_levenshtein, row[1])
levenshtein_pd = pd.DataFrame({'indirizzo': strings, 'ratio': ratios_levenshtein})
levenshtein_pd

,indirizzo,ratio
0,"Via Aurelio Saffi, 26 20123 Milano",100.0
1,"Via Aurelio Saffi, 36 20123 Milano",97.0
2,Via Aurelio Saffi 26 Milano,89.0
3,"Via A. Maffi, 26 20123 Milano",86.0
4,Via A. Saffi 26 Milano,75.0
5,Via Aurelio Saffi numero civico 26 Milano,75.0
6,Via Saffi Aurelio 26 Milano,69.0


In [62]:
# The token sort approach involves tokenizing the string in question, 
# sorting the tokens alphabetically, and then joining them back into a string.
# We then compare the transformed strings with a simple ratio(levenshtein) 
token_sort_ratio = process.extract(s1, options, scorer=fuzz.token_sort_ratio, limit=7)
# token_sort_ratio
strings = []
ratios_token_sort_ratio = []
for index, row in enumerate(token_sort_ratio):
    strings = np.append(strings, row[0]) 
    ratios_token_sort_ratio = np.append(ratios_token_sort_ratio, row[1])
token_sort_ratio_pd = pd.DataFrame({'indirizzo': strings, 'ratio': ratios_token_sort_ratio})
token_sort_ratio_pd

,indirizzo,ratio
0,"Via Aurelio Saffi, 26 20123 Milano",100.0
1,"Via Aurelio Saffi, 36 20123 Milano",97.0
2,Via Aurelio Saffi 26 Milano,90.0
3,Via Saffi Aurelio 26 Milano,90.0
4,Via A. Saffi 26 Milano,78.0
5,Milano Via A. Saffi 26,78.0
6,Via Aurelio Saffi numero civico 26 Milano,73.0


In [63]:
# 1. Take the ratio of the two processed strings (fuzz.ratio)
# 2. Run checks to compare the length of the strings
#     * If one of the strings is more than 1.5 times as long as the other
#       use partial_ratio comparisons - scale partial results by 0.9
#       (this makes sure only full results can return 100)
#     * If one of the strings is over 8 times as long as the other
#       instead scale by 0.6
# 3. Run the other ratio functions
#     * if using partial ratio functions call partial_ratio,
#       partial_token_sort_ratio and partial_token_set_ratio
#       scale all of these by the ratio based on length
#     * otherwise call token_sort_ratio and token_set_ratio
#     * all token based comparisons are scaled by 0.95
#       (on top of any partial scalars)
# 4. Take the highest value from these results
#    round it and return it as an integer.
weighted_ratio = process.extract(s1, options, scorer=fuzz.WRatio, limit=7)
# weighted_ratio
strings = []
ratios_weighted_ratio = []
for index, row in enumerate(weighted_ratio):
    strings = np.append(strings, row[0]) 
    ratios_weighted_ratio = np.append(ratios_weighted_ratio, row[1])
weighted_ratio_pd = pd.DataFrame({'indirizzo': strings, 'ratio': ratios_weighted_ratio})
weighted_ratio_pd

,indirizzo,ratio
0,"Via Aurelio Saffi, 26 20123 Milano",100.0
1,"Via Aurelio Saffi, 36 20123 Milano",97.0
2,Via Aurelio Saffi 26 Milano,95.0
3,Via Saffi Aurelio 26 Milano,95.0
4,Via A. Saffi 26 Milano,86.0
5,Milano Via A. Saffi 26,86.0
6,Via Aurelio Saffi numero civico 26 Milano,86.0


In [64]:
summary_pd = pd.DataFrame({'indirizzo': strings, 'levenshtein ratio': ratios_levenshtein, 'token sort ratio': ratios_token_sort_ratio, 'weighted ratio': ratios_weighted_ratio})
summary_pd

,indirizzo,levenshtein ratio,token sort ratio,weighted ratio
0,"Via Aurelio Saffi, 26 20123 Milano",100.0,100.0,100.0
1,"Via Aurelio Saffi, 36 20123 Milano",97.0,97.0,97.0
2,Via Aurelio Saffi 26 Milano,89.0,90.0,95.0
3,Via Saffi Aurelio 26 Milano,86.0,90.0,95.0
4,Via A. Saffi 26 Milano,75.0,78.0,86.0
5,Milano Via A. Saffi 26,75.0,78.0,86.0
6,Via Aurelio Saffi numero civico 26 Milano,69.0,73.0,86.0
